In [1]:
import os
import json
import time
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from io import StringIO
from datetime import datetime
import glob


In [73]:
# API Keys provided by PurpleAir(c)
key_read  = '6CF9C562-6AEB-11EF-95CB-42010A80000E'

# Sleep Seconds
sleep_seconds = 3 # wait sleep_seconds after each query

# Data download period. Enter Start and end Dates
bdate = '03-18-2022'
edate = '03-24-2022'

# Average_time. The desired average in minutes, one of the following: 0 (real-time), 
# 10 (default if not specified), 30, 60, 360 (6 hour), 1440 (1 day)
average_time=10 # or 10  or 0 (Current script is set only for real-time, 10, or 60 minutes data)

data_dir = "data"
processed_dir = "processed"

In [79]:
# Function to update the log file with the last download date and sensor info
def update_sensor_index_log(log_file_path, sensor_list, last_download_date):
    log_data = {}

    # Check if the log file exists, and load existing data if available
    if os.path.exists(log_file_path):
        with open(log_file_path, 'r') as log_file:
            log_data = json.load(log_file)

    # Update the log with the last download date
    log_data["sensors_index.csv file last updated"] = str(last_download_date)

    # Add or update each sensor in the log
    for sensor in sensor_list:
        if str(sensor) not in log_data:
            log_data[str(sensor)] = {
                "min_date": None,
                "max_date": None,
                "url_issue": [],
                "no_data": [],
                "in_jay": []
            }
    
    # Write the updated log to a JSON file
    with open(log_file_path, 'w') as log_file:
        json.dump(log_data, log_file, indent=4)
    
    print(f"{log_file_path} successfully updated.")

In [80]:
# Function to get sensors from the API
def get_sensors(key_read):
    # PurpleAir API URL
    root_url = 'https://api.purpleair.com/v1/sensors/'
    
    # fields to get
    fields_list = ['name', 'location_type', 'latitude', 'longitude', 'altitude', 
                   'position_rating', 'uptime', 'last_seen', 'last_modified', 'date_created']
    
    # Build the fields parameter for the API call
    fields_api_url = '&fields=' + '%2C'.join(fields_list)
    
    # Final API URL
    api_url = root_url + f'?api_key={key_read}' + fields_api_url
    
    print(api_url)
    
    # Getting data
    response = requests.get(api_url)
    if response.status_code == 200:
        json_data = json.loads(response.content)
        df = pd.DataFrame.from_records(json_data["data"])
        df.columns = json_data["fields"]
    else:
        json_data = json.loads(response.content)
        print("Error description:", json_data["description"])
        raise requests.exceptions.RequestException("Failed to fetch sensor data.")
    
    # ----------------------- clean sensor index df
    # Convert UNIX timestamps to readable date format
    df['last_modified'] = pd.to_datetime(df['last_modified'], unit='s')
    df['date_created'] = pd.to_datetime(df['date_created'], unit='s')
    df['last_seen'] = pd.to_datetime(df['last_seen'], unit='s')
    
    # ----------------------- create new column 'us': us = 1 if a sensor is in US; otherwise = 0
    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)

    # Load U.S. boundaries from a GeoJSON file
    us_shp = gpd.read_file(os.path.join(data_dir, "us_shp.json"))

    # Ensure the CRS matches
    us_shp.to_crs(gdf.crs, inplace=True)

    # Perform a spatial join to check if the points are in the U.S.
    gdf_us = gpd.sjoin(gdf, us_shp, how="left", predicate="intersects")

    # Add 'us' flag: if the point is in the U.S.
    gdf_us['us'] = gdf_us['index_right'].notnull().astype(int)

    # Drop unnecessary columns
    gdf_sensors = gdf_us.drop(columns=['index_right', 'AFFGEOID', 'GEOID', 'NAME'])

    # Directory to store the file
    os.makedirs(processed_dir, exist_ok=True)  # Create directory if it doesn't exist
    
    filename = os.path.join(processed_dir, 'sensors_index.csv')
    # Define file path and save the CSV file
    gdf_sensors.to_csv(filename, index=False, header=True)

    print(f"{len(gdf_sensors)} sensors successfully extracted and stored in: {filename}")

    # extract sensor_list
    sensor_list = gdf_sensors['sensor_index'].tolist()
    
    # Update the download log
    today = datetime.now().date()

    log_file_path = 'sensor_log.json'
    update_sensor_index_log(log_file_path, sensor_list, today)
    
    return gdf_sensors

In [81]:
get_sensors(key_read=key_read)

https://api.purpleair.com/v1/sensors/?api_key=6CF9C562-6AEB-11EF-95CB-42010A80000E&fields=name%2Clocation_type%2Clatitude%2Clongitude%2Caltitude%2Cposition_rating%2Cuptime%2Clast_seen%2Clast_modified%2Cdate_created
Error description: Payment is required to make this api call.


RequestException: Failed to fetch sensor data.

In [83]:
# extract sensor_list
sensor_list = pd.read_csv("processed/sensors_index.csv")['sensor_index'].tolist()

# Update the download log
today = datetime.now().date()

log_file_path = 'sensor_log.json'
update_sensor_index_log(log_file_path, sensor_list, today)

sensor_log.json successfully updated.


### Sensor data fields

| Field           | Type     | Description                                                                                                  |
|-----------------|----------|--------------------------------------------------------------------------------------------------------------|
| sensor_index    | Integer  | The sensor's index. Can be used to add a sensor to a group or view its details.                               |
| last_modified   | Long     | The UNIX time stamp from the last time the device was modified by the registration form.                      |
| date_created    | Long     | The UNIX time stamp from when the device was created.                                                         |
| last_seen       | Long     | The UNIX time stamp of the last time the server received data from the device.                                |
| name            | String   | The name given to the sensor from the registration form and used on the PA map.                               |
| location_type   | Integer  | The location type. Possible values are: 0 = Outside or 1 = Inside.                                            |
| uptime          | Integer  | The time in minutes since the firmware started as last reported by the sensor.                                |
| position_rating | Integer  | A 'star' rating of position accuracy. 0 stars is nowhere near the claimed location whereas 5 stars is close to the map location as indicated by the latitude and longitude values. |
| latitude        | Number   | The latitude position value for the sensor.                                                                   |
| longitude       | Number   | The longitude position value for the sensor.                                                                  |
| altitude        | Number   | The altitude for the sensor's location in feet.                                                               |






In [ ]:
sensors_index = pd.read_csv('data/sensors_index.csv')
sensors_index.head()

In [17]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    sensors_index, 
    geometry=gpd.points_from_xy(sensors_index.longitude, sensors_index.latitude),
    crs=4326   
)

# Read the GeoJSON file back into a GeoDataFrame
us_shp = gpd.read_file("data/us_shp.json")

# set same crs
us_shp.to_crs(gdf.crs, inplace=True)

# Perform a spatial join to see if points in gdf intersect with the U.S. geometry
gdf_us = gpd.sjoin(gdf, us_shp, how="left", predicate="intersects")

# Create the 'us' flag column: if 'index_right' is not null, the point is in the U.S.
gdf_us['us'] = gdf_us['index_right'].notnull().astype(int)

# Drop the 'index_right' column and keep the original gdf with the flag
gdf_sensorlist = gdf_us.drop(columns=['index_right', 'AFFGEOID', 'GEOID', 'NAME', 'Unnamed: 0'])

gdf_sensorlist.to_csv('data/sensors_index.csv')

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load a base map of the world
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Define colors for location types and US sensors
colors = {0: 'blue', 1: 'red'}  # Blue and Red for location types
us_color = 'green'  # Green for US sensors

# Plot the base map
fig, ax = plt.subplots(figsize=(12, 8))
world.plot(ax=ax, color='lightgrey')

# Plot sensor locations by location type
for location_type, color in colors.items():
    subset = gdf_sensorlist[gdf_sensorlist['location_type'] == location_type]
    subset.plot(ax=ax, color=color, markersize=50, alpha=0.6, label=f'Location Type {location_type}')

# Plot sensors in the US
us_sensors = gdf_sensorlist[gdf_sensorlist['us'] == 1]  # Assuming 'us' column is binary: 1 for US, 0 for non-US
us_sensors.plot(ax=ax, color=us_color, markersize=50, alpha=0.6, label='US Sensors')

# Add legend
plt.legend(title='Location Type and US')

# Add titles and labels
plt.title('Station Points on Global Map by Location Type and US Sensors')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()

In [ ]:
# Display the total number of available sensors
print("Total number of available sensors:", len(sensors_index))

# Display the most recent and oldest sensors based on the 'date_created' timestamp
print("Most recent sensor created on:", pd.to_datetime(max(sensors_index["date_created"])).strftime('%Y-%m-%d %H:%M:%S'))
print("Oldest sensor created on:", pd.to_datetime(min(sensors_index["date_created"])).strftime('%Y-%m-%d %H:%M:%S'))

# Display the available sensors by position rating, sorted by count
print("\nAvailable sensors by position rating:")
print(sensors_index['position_rating'].value_counts().sort_values())

# Display the available sensors by location type (0 = outdoor, 1 = indoor)
print("\nAvailable sensors by location type (0 = outdoor, 1 = indoor):")
print(sensors_index['location_type'].value_counts())

print("\nAvailable sensors by the last time of year the server received data from the device:")
print(pd.to_datetime(sensors_index['last_seen']).dt.strftime('%Y').value_counts().sort_values())

# Create histogram of the 'uptime' column
plt.figure(figsize=(10, 6))
plt.hist(sensors_index['uptime'], bins=40, edgecolor='black')
plt.title('Histogram of Uptime')
plt.xlabel('Uptime (minutes)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

To do list: 
1. do we need sensors that is not in Jay's sensor list?
    - Jay's number of sensors are about 26000. Do we care about matching the current sensors with already downloaded sensors as much as possible or should we download available sensors and match it later?
    - also, based on jay's code, seems like he downloaded only us_indoor sensors data? is that correct? I thought we want to download all sensors across globally?

    - **Answer**: We don’t want to duplicate downloading data Jay already downloaded but we do want to download any available sensor data even if that sensor is not in Jay’s dataset (eg if it came online only recently). Also you’re right both that we want all sensors and that Jay started on US indoor. That was just a way for Jay to order the downloads he was ultimately planning on downloading everything just picked US indoor as the first group and then never got around to other groups.

2. do we need sensors whose position rating is doubtful? 

    - **Answer**: I think we can ignore position_rating for now and download everything. From reading about it that variable sounds like a potentially unreliable measure of position accuracy.
 
3. do we need sensors who stopped communicating to the PA prior years to 2024?

    - **Answer**: Yes in general we want all data from sensors even if they stopped communicating before 2024. That being said I think we can set some filter for minimum time online. Like maybe we only include sensors that were/are online for at least 180 days (~6 months). I don’t think we need to worry about monitors with only a few days of data available. I think we can check this from diff between first_seen and last_seen. Doesn’t need to be perfect.

4. data we have runs from Jan 1, 2021 to Dec 31, 2023. 

(From Jay's work) Prioritization of PA: 

- indoor US
- outdoor US 
- indoor international
- outdoor international

Variable we want: 
- ‘pm2.5_atm_a’
- ‘pm2.5_atm_b’
- ‘pm2.5_cf_1_a’
- ‘pm2.5_cf_1_b’
- ‘humidity’
- ‘temperature’

In [45]:
def get_sensorslist():
    """Retrieve sensor indexes based on whether they are US or not."""
    
    if os.path.exists(os.path.join(processed_dir, 'sensors_index.csv')):
        sensors_index = pd.read_csv(os.path.join(processed_dir, 'sensors_index.csv'))
    else:
        sensors_index = get_sensors(key_read = key_read)
        
    # replace this with get_sensor function later!
    # all_sensorlist = gdf_sensorlist['sensor_index'].tolist()
        
    # subset us_indoor and us_oudoor sensors
    # us_sensorlist = sensors_index[sensors_index['us'] == 1]['sensor_index'].tolist()
    us_indoor_sensorlist = sensors_index[(sensors_index['us'] == 1) & (sensors_index['location_type'] == 1)]['sensor_index'].tolist()
    us_outdoor_sensorlist = sensors_index[(sensors_index['us'] == 1) & (sensors_index['location_type'] == 0)]['sensor_index'].tolist()
    
    # non_us_indoor and non_us_outdoor sensors
    nonus_sensorlist = sensors_index[sensors_index['us'] == 0]['sensor_index'].tolist()

    sensor_dict = {
        "us_indoor": us_indoor_sensorlist, 
        "us_outdoor": us_outdoor_sensorlist,
        "non_us": nonus_sensorlist
        }
    
    return sensor_dict

In [57]:
def calculate_pa_points(n_sensors, begin_time, end_time):
    
    # point each request
    rows = 714
    columns = 8 + 2
    point_per_request = int(rows*columns)
    
    # find number of days from begin_time: 01-01-2021 to end_time: 12:31:2023
    # Convert strings to datetime objects
    begin_date = datetime.strptime(begin_time, '%m-%d-%Y')
    end_date = datetime.strptime(end_time, '%m-%d-%Y')

    # Calculate the difference in days
    n_days = (end_date - begin_date).days
    print(n_days)
    total_points_per_sensor = n_days/5*point_per_request
    
    total_points_n_sensors = int(total_points_per_sensor) * int(n_sensors)
    
    return int(total_points_n_sensors)

sensors_dict = get_sensorslist()
us_indoor_sensorlist = sensors_dict["us_indoor"]
us_outdoor_sensorlist = sensors_dict["us_outdoor"]
nonus_sensorlist = sensors_dict["non_us"]

# Define the start and end dates
begin_time = '01-01-2021'
end_time = '12-31-2023'

us_indoor_points = calculate_pa_points(len(us_indoor_sensorlist), begin_time=begin_time, end_time = end_time) 
us_outdoor_points = calculate_pa_points(len(us_outdoor_sensorlist), begin_time=begin_time, end_time = end_time) 
non_us_points = calculate_pa_points(len(nonus_sensorlist), begin_time=begin_time, end_time = end_time) 

# print("Length of all sensors:", len(all_sensorlist))
# print("Length of us sensors:", len(us_sensorlist))
print(f"Length of us_indoor sensors: {len(us_indoor_sensorlist)} ({us_indoor_points/1e9} billion points) ")
print(f"Length of us_outdoor sensors: {len(us_outdoor_sensorlist)} ({us_outdoor_points/1e9} billion points) ")
print(f"Length of non-us sensors: {len(nonus_sensorlist)} ({non_us_points/1e9} billion points)")

# print("Length of us indoor sensors that are already downloaded:", len(us_indoor_jay))

1094
1094
1094
Length of us_indoor sensors: 5743 (8.971898376 billion points) 
Length of us_outdoor sensors: 16626 (25.973669232 billion points) 
Length of non-us sensors: 4472 (6.986301504 billion points)


In [47]:
def create_pa_datelist(average_time, bdate, edate):
    """Create a list of date ranges for the historical data period."""
    
    # Dates of Historical Data period
    begindate = datetime.fromisoformat(bdate)
    enddate = datetime.fromisoformat(edate)
    
    # Download days based on average
    if average_time == 60:
        datelist = pd.date_range(begindate, enddate, freq='14d')  # 14 days of data
    else:
        datelist = pd.date_range(begindate, enddate, freq='5d')   # 5 days of data

    # Reverse to get data from end date to start date
    datelist = datelist.tolist()
    datelist.reverse()
    
    # Convert to required format
    date_list = [dt.strftime('%Y-%m-%dT%H:%M:%SZ') for dt in datelist]
    
    return date_list

In [2]:
def get_historicaldata(sensors_list, bdate, edate, average_time, data_dir, processed_dir, key_read, sleep_seconds):
    """Download historical data for given sensors."""
    
    # Load the download log file
    log_file_path = 'sensor_log.json'
    with open(log_file_path, 'r') as log_file:
        log_data = json.load(log_file)
    
    # Historical API URL: for multiple sensors
    root_api_url = 'https://api.purpleair.com/v1/sensors/'
    
    # Average time parameter for API
    average_api = f'&average={average_time}'
    
    # Create the fields API URL
    fields_list = ['pm2.5_atm_a', 'pm2.5_atm_b', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b', 'humidity', 'temperature']
    fields_api_url = '&fields=' + '%2C'.join(fields_list)
    
    print("CHECK POINT 1: Fields API URL")
    print(fields_api_url)
    
    # Retrieve sensor index for us_indoor
    us_indoor_sensors = pd.read_csv(os.path.join(data_dir, "indoor_us_jay.csv"))
    us_indoor_sensorlist_jay = us_indoor_sensors['sensor_index'].tolist()

    print("CHECK POINT 2: Indoor Sensor List")
    print(us_indoor_sensorlist_jay)
    
    # Generate date list for all sensors
    date_list = create_pa_datelist(average_time, bdate, edate)
    
    print("CHECK POINT 3: Generated Date List")
    print(date_list)
    
    # Generate date list to skip data (from 2021-01-01 to 2023-12-31)
    check_datelist = create_pa_datelist(average_time, '2021-01-01T00:00:00+00:00', '2023-12-31T00:00:00+00:00')
    
    print("CHECK POINT 4: Date List for Skipping")
    print(check_datelist)
    
    len_datelist = len(date_list) - 1
    
    # Process each sensor
    for sensor in sensors_list:
        
        # create new folder for a sensor
        sensor_folder = os.path.join(processed_dir, f"sensorID_{sensor}")
        os.makedirs(sensor_folder, exist_ok=True)  # Create sensor folder if it doesn't exist
        
        hist_api_url = root_api_url + f'{sensor}/history/csv?api_key={key_read}'
        
        # Check if any file exists in the folder: returns a list of files
        existing_files = glob.glob(f'{sensor_folder}/*.csv')
        
        print("CHECK POINT 5: Existing Files")
        print(existing_files)
        
        # If any file exists, use the latest file
        if existing_files:
            # Read the existing file
            existing_df = pd.read_csv(existing_files[0])
        else:
            print(f"No existing files for sensor {sensor}. Starting new file.")
            existing_df = pd.DataFrame()  # No existing data

        # Create start and end date API URL
        for index, date in enumerate(date_list):
            time.sleep(sleep_seconds)  # Throttle API requests
            
            if index < len_datelist:
                
                # Skip if sensor is in us_indoor and the date is in 2021-2023
                if sensor in us_indoor_sensorlist_jay and min(check_datelist) <= date <= max(check_datelist):
                    log_data[str(sensor)]["in_jay"].append("yes")
                    print(f"Skipping download for {sensor} from {date_list[index+1]} to {date} (us_indoor sensor)")
                    continue

                # Download data for PA
                print(f'Downloading for PA: {sensor} for Dates: {date_list[index+1]} and {date}.')
                dates_api_url = f'&start_timestamp={date_list[index+1]}&end_timestamp={date}'
                
                api_url = hist_api_url + dates_api_url + average_api + fields_api_url
                                    
                try:
                    response = requests.get(api_url)
                    response.raise_for_status()  # Raises an exception for 4xx/5xx responses
                    df = pd.read_csv(StringIO(response.text), sep=",", header=0)
                
                # Try break for any API related error 
                # or break when only payment issue with API
                except requests.exceptions.RequestException as e:
                    try:
                        json_data = json.loads(response.content)
                        if json_data["description"] == "Payment is required to make this api call.":
                            print(json_data["description"])
                            break
                        else: 
                            log_data[str(sensor)]["URL_issue"].append(json_data["description"])
                    except:
                        log_data[str(sensor)]["URL_issue"].append(f"Request failed: {str(e)}")
                    continue
                
                except pd.errors.EmptyDataError:
                    log_data[str(sensor)]["no_data"].append(f"{date_list[index+1]} to {date}")
                    continue
                
                except Exception as e:
                    print(f"Unexpected error for sensor {sensor}: {e}")
                    continue

                if df.empty:
                    log_data[str(sensor)]["no_data"].append(f"{date_list[index+1]} to {date}")
                    continue
                
                # Process DataFrame
                df['time_stamp'] = pd.to_datetime(df['time_stamp'])
                df = df.drop_duplicates().sort_values(by='time_stamp')

                # Append the new data to the existing DataFrame if it exists
                if not existing_df.empty:
                    df = pd.concat([existing_df, df], ignore_index=True)
                    df['time_stamp'] = pd.to_datetime(df['time_stamp'])
                    df = df.drop_duplicates().sort_values(by='time_stamp')

                # Get the minimum and maximum date from the DataFrame
                min_date = df['time_stamp'].min()
                max_date = df['time_stamp'].max()

                # Convert the dates from ISO format to the desired string format
                min_date = min_date.strftime("%Y_%m_%d")
                max_date = max_date.strftime("%Y_%m_%d")

                # Parquet file path (corrected extension to .parquet)
                csv_file = f'{sensor_folder}/sensorID_{sensor}_{min_date}_{max_date}.csv'

                # Save the DataFrame to Parquet file
                df.to_parquet(csv_file, index=False)
                print(f"Data saved to: {csv_file}")
    
    # Save the updated log file at the end of the process
    with open(log_file_path, 'w') as log_file:
        json.dump(log_data, log_file, indent=4)

In [56]:
# Data download period
from datetime import datetime

bdate = '2021-01-01T00:00:00+00:00' 
edate = '2021-01-12T00:00:00+00:00'

# edate = datetime.now().strftime('%Y-%m-%d') + 'T00:00:00+00:00'
# sensors_list = get_sensors(us_indoor=False)
# sample_sensors = [131255, 131365, 469, 53, 77, 182]
sample_sensors = [131255, 182]
result = get_historicaldata(sensors_list=sample_sensors, bdate= bdate, edate=edate, average_time=10)

[131255, 182]
CHECK POINT 1:
&fields=pm2.5_atm_a%2Cpm2.5_atm_b%2Cpm2.5_cf_1_a%2Cpm2.5_cf_1_b%2Chumidity%2Ctemperature
CHECK POINT 2:
[131255, 131365, 469, 131615, 131853, 820, 1004, 132135, 132197, 132469, 132481, 132763, 132839, 132841, 132853, 132929, 132951, 133047, 133131, 133251, 133283, 133373, 133497, 133519, 133666, 133664, 2624, 2628, 133728, 133736, 133742, 133740, 133750, 133768, 133778, 133868, 133878, 133880, 133896, 133942, 133964, 3011, 134084, 134092, 134102, 134104, 134112, 134120, 134128, 134140, 134150, 3118, 134252, 3253, 134366, 134370, 134374, 134382, 134380, 134388, 134398, 134396, 134400, 3497, 134722, 134746, 134758, 134760, 134772, 134806, 134814, 134894, 3862, 3882, 134966, 135004, 135012, 135016, 135020, 3951, 135024, 135030, 135028, 135044, 135050, 135080, 135132, 135156, 135196, 135214, 135222, 135220, 135226, 135230, 135380, 135416, 135438, 4368, 135440, 135492, 4450, 4476, 135548, 135558, 4551, 135624, 4635, 135720, 135734, 135736, 135756, 135766, 135772

In [ ]:
result = pd.read_csv('data/sensorsID_182_2021_01_01_2021_01_06.csv')
result

The script will download each sensor for dates with five days of increments. Each download will have rows = 720 (144 for each day * 5 days increment) and columns = 8 (+2 as humidity and temperature 2 points each)

for each sensor: 
- point each day = 720*10 = 7200
- days total from 2021 to today = 3 years, 8 months and 5 days ago, which is 1,344 days
- total points per sensor = 1344 day * 7200 points per day = 9676800 point per sensor

For 26841 sensors:
- total points = 9676800 points per sensor * 26841 sensor = 2.59734989 * 10^11 = 259.73 billion points


| time_stamp        | sensor_index | humidity | temperature | pm2.5_atm_a | pm2.5_atm_b | pm2.5_cf_1_a | pm2.5_cf_1_b |
|-------------------|--------------|----------|-------------|-------------|-------------|--------------|--------------|
| 3/1/2023 0:00     | 15237        | 37       | 72.6        | 0.666       | 0.418       | 0.666        | 0.418        |
| 3/1/2023 0:10     | 15237        | 37       | 72          | 0.496       | 0.340       | 0.496        | 0.340        |


### To do list (sep 5, 2024): 

- i don't want to download data that RA did
    - us indoor from 2021 jan 1 to 2023 dec 31
- first get all sensor index that is in us indoor
- secod get all sensor index that is not us indoor (outside of us indoor)

1. Get Sensors:
    - `get_sensors()` retrieves sensor indexes based on whether they are part of us_indoor or not. 
2. Determine Date Ranges:
    - If the sensor is in `us_indoor` and the date range is from `January 1, 2021`, to `December 31, 2023`, skip the download.
3. Download Data:
    - For sensors not in the `us_indoor` category or for dates outside the specified range, proceed with the download logic.
4. Update Download Records:
    - After downloading data, the last downloaded date is updated to ensure that the next run continues from where it left off.

### to do list (sep 6, 2024)

- Create download_records: begindate, and lastdate per sensor?
- Use PostSQL or OAK?
     - try OAK 
     - download only US indoor and outdoor from 01/01/2021 - 12/31/2023 first

### to do list (sep 9, 2024 - now)

1. Create PA database on OAK
    - Upload downloaded us_indoor PA data to OAK PA database
        - Write a script that:
            - restructures the current `sensor`->`year`->`month` into `sensor-startdate-enddate`
            - stores it on OAK PA database

2. Download global sensors first (then, us indoor and outdoor) from 2021-2023 (skipping us indoors that are already downloaded)
    - each sensor:
        - download 5 days of 10 min PA data
        - store 5 days data to database on OAK by `sensor-startdate-enddate` file


